# What is JuliaDB?

We recognized the need for an all-Julia, end-to-end tool that can

- **Load multi-dimensional datasets quickly and incrementally.**
- **Index the data and perform filter, aggregate, sort and join operations.**
- **Save results and load them efficiently later.**
- **Use Julia's built-in parallelism to fully utilize any machine or cluster.**

We built JuliaDB to fill this void.

# Sales Pitch

- Be fast
- Work with multi-file datasets
- Handles heavy lifting of distributed computing
- Batteries included
- **No glue code**

![](languages.png)

In [2]:
using JuliaDB

# Data Structures

- `table`
- `ndsparse`
- Distributed versions are also available

In [3]:
x = rand(Bool, 10)
y = rand('a':'b', 10)
z = randn(10);

## `table` vs. `ndsparse`

In [4]:
t = table(@NT(x=x, y=y, z=z); pkey = [:x, :y])

Table with 10 rows, 3 columns:
x     y    z
────────────────────
true  'a'  -0.775464
true  'a'  -0.631119
true  'a'  -1.46226
true  'a'  -0.330762
true  'a'  0.288622
true  'b'  0.333483
true  'b'  2.73925
true  'b'  0.411718
true  'b'  -1.24639
true  'b'  0.927941

In [5]:
nd = ndsparse(@NT(x=x, y=y), z)

2-d NDSparse with 10 values (Float64):
x     y   │
──────────┼──────────
true  'a' │ -0.775464
true  'a' │ -0.631119
true  'a' │ -1.46226
true  'a' │ -0.330762
true  'a' │ 0.288622
true  'b' │ 0.333483
true  'b' │ 2.73925
true  'b' │ 0.411718
true  'b' │ -1.24639
true  'b' │ 0.927941

In [8]:
t[1]

(x = true, y = 'a', z = -0.7754640116187353)

In [68]:
nd[false, 'a']

-0.4232329675776797

# Loading Data

In [9]:
loadtable("stocksample")

Table with 56023 rows, 7 columns:
Date        Open     High     Low      Close    Volume    OpenInt
─────────────────────────────────────────────────────────────────
1984-09-07  0.42388  0.42902  0.41874  0.42388  23220030  0
1984-09-10  0.42388  0.42516  0.41366  0.42134  18022532  0
1984-09-11  0.42516  0.43668  0.42516  0.42902  42498199  0
1984-09-12  0.42902  0.43157  0.41618  0.41618  37125801  0
1984-09-13  0.43927  0.44052  0.43927  0.43927  57822062  0
1984-09-14  0.44052  0.45589  0.44052  0.44566  68847968  0
1984-09-17  0.45718  0.46357  0.45718  0.45718  53755262  0
1984-09-18  0.45718  0.46103  0.44052  0.44052  27136886  0
1984-09-19  0.44052  0.44566  0.43157  0.43157  29641922  0
1984-09-20  0.43286  0.43668  0.43286  0.43286  18453585  0
1984-09-21  0.43286  0.44566  0.42388  0.42902  27842780  0
1984-09-24  0.42902  0.43157  0.42516  0.42516  22033109  0
⋮
2017-10-27  319.75   324.59   316.66   320.87   6970118   0
2017-10-30  319.18   323.78   317.25   320.08   4254

# Saving Data